In [1]:
%load_ext autoreload
%autoreload 2

import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(os.path.dirname(os.path.dirname(current_dir)))
sys.path.insert(0, os.path.join(parent_dir))

In [2]:
import pickle
import pandas as pd

from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor

from coord2vec.Noam_Adir.pipeline.preprocess import generic_clean_col, ALL_MANHATTAN_FILTER_FUNCS_LIST
from coord2vec.Noam_Adir.pipeline.base_pipeline import extract_geographical_features, train_models, plot_scores


/data/home/morpheus/coord2vec_noam/coord2vec/common/parallel/multiproc_util.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# features_bundle = karka_bundle_features
# use_full_dataset = True
# batch_size = 10000
# clean_funcs = [clean_floor_col, clean_constructionTime_col]
n_catboost_iter = 150

In [4]:
def load_manhatan_data_from_pickel(file_name="manhattan_house_prices.pkl"):
    manhatan_df = pickle.load( open( pickle_file_name, "rb" ) )
    manhatan_df["coord"] = manhatan_df.apply(lambda row: tuple(row[['lon', 'lat']].values), axis=1)
    return manhatan_df

In [5]:
def get_non_repeating_coords(full_df:pd.DataFrame):
    df = full_df.copy()
    if not "coord_id" in df.columns:
        coord_2coordid = {coord:i for i,coord in enumerate(manhatan_df["coord"].unique())}
        df["coord_id"] = manhatan_df.apply(lambda row: coord_2coordid[row["coord"]], axis=1)
    apeared_coord_id = {}
    coord_ids = df["coord_id"]
    for inx in df.index:
        if coord_ids[inx] in apeared_coord_id:
            df = df.drop([inx])
        else:
            apeared_coord_id[coord_ids[inx]]=True
#     print("test sould be one:", len(set(apeared_coord_id.values())))
    return  df
    

In [6]:
pickle_file_name = "manhattan_house_prices.pkl"
manhatan_df = pickle.load( open( pickle_file_name, "rb" ) )
manhatan_df["coord"] = manhatan_df.apply(lambda row: tuple(row[['lon', 'lat']].values), axis=1)

manhatan_df = generic_clean_col(manhatan_df, ALL_MANHATTAN_FILTER_FUNCS_LIST)
cleaned_features = manhatan_df[['sold','priceSqft', 'numBedrooms', 'numBathrooms', 'sqft', 'coord']]
all_features = extract_geographical_features(cleaned_features)

In [7]:
all_features

,sold,priceSqft,numBedrooms,numBathrooms,sqft,coord,coord_id,distance_to_major_road_100m,length_of_major_road_100m,area_of_nearest_major_road_100m,...,area_of_nearest_building_50m,distance_to_building_100m,number_of_building_100m,area_of_building_100m,area_of_nearest_building_100m,distance_to_building_250m,number_of_building_250m,area_of_building_250m,area_of_nearest_building_250m,area_of_building_0m
0,900000.0,1384,1,1.0,650,"(-73.986817, 40.736249)",0,22.872667,465.826034,75.006598,...,1397.884593,0.000000,80.0,2630.299381,1397.884593,0.000000,269.0,10678.102122,1397.884593,129.866647
1,825000.0,1000,1,1.0,825,"(-73.97482099999999, 40.791194)",1,27.728680,2373.860017,1734.348480,...,777.107354,0.000000,47.0,1658.078754,777.107354,0.000000,256.0,8644.938566,777.107354,72.195035
2,540000.0,947,1,1.0,570,"(-73.968053, 40.760779)",2,29.472751,1387.348456,72.088905,...,918.006024,0.000000,37.0,2144.205749,918.006024,0.000000,206.0,11855.526306,918.006024,85.284841
3,1700000.0,1446,2,2.0,1175,"(-73.989811, 40.73043)",3,40.134080,793.254435,347.829569,...,1393.991316,0.000000,33.0,2111.188242,1393.991316,0.000000,263.0,9860.480359,1393.991316,129.504953
4,825000.0,916,2,1.0,900,"(-73.972274, 40.795986)",4,29.690490,1694.095759,723.844180,...,558.380436,0.000000,36.0,1626.600631,558.380436,0.000000,190.0,8049.831443,558.380436,51.874808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16002,850000.0,1000,1,1.0,850,"(-73.984585, 40.780136999999996)",2035,25.115671,2057.076778,1152.925813,...,708.219453,0.000000,48.0,1701.318135,708.219453,0.000000,268.0,7996.404018,708.219453,65.795193
16003,999999.0,1176,1,1.0,850,"(-73.97379000000001, 40.793639)",4376,46.423191,604.256062,319.116799,...,526.422910,1.557385,53.0,1571.557034,526.422910,1.557385,220.0,8414.254623,526.422910,0.000000
16004,1410000.0,1046,3,2.0,1350,"(-73.958597, 40.766527)",5488,100.000000,0.000000,0.000000,...,742.309093,0.000000,69.0,2032.903064,742.309093,0.000000,228.0,10673.276929,742.309093,68.962197
16005,565000.0,1412,1,1.0,400,"(-74.002088, 40.732318)",4242,100.000000,0.000000,0.000000,...,182.288808,0.000000,126.0,1995.697937,182.288808,0.000000,566.0,9402.313632,182.288808,16.935043


## train on all of the data

In [8]:
models = [LinearRegression(),
          CatBoostRegressor(iterations=n_catboost_iter, learning_rate=1, depth=3)]
y_col = "sold"
training_cache = train_models(models=models, all_features=all_features, y_col=y_col)

0:	learn: 936131.5988813	total: 486ms	remaining: 1m 12s
1:	learn: 840590.2918822	total: 491ms	remaining: 36.3s
2:	learn: 677265.2812651	total: 493ms	remaining: 24.2s
3:	learn: 617500.9906573	total: 495ms	remaining: 18.1s
4:	learn: 577572.6007487	total: 498ms	remaining: 14.4s
5:	learn: 547960.7800562	total: 502ms	remaining: 12s
6:	learn: 522495.7143496	total: 506ms	remaining: 10.3s
7:	learn: 508931.5796127	total: 510ms	remaining: 9.06s
8:	learn: 493145.6343280	total: 517ms	remaining: 8.1s
9:	learn: 482215.7763370	total: 522ms	remaining: 7.3s
10:	learn: 456068.7064289	total: 526ms	remaining: 6.64s
11:	learn: 421694.3426901	total: 531ms	remaining: 6.1s
12:	learn: 392624.3555131	total: 536ms	remaining: 5.65s
13:	learn: 380319.5461492	total: 540ms	remaining: 5.25s
14:	learn: 367630.5002851	total: 544ms	remaining: 4.89s
15:	learn: 361172.3678129	total: 547ms	remaining: 4.58s
16:	learn: 350966.2728904	total: 553ms	remaining: 4.33s
17:	learn: 343653.4940215	total: 559ms	remaining: 4.1s
18:	lea

In [9]:
plot_scores(training_cache)

mean price -  1593800.5824587706
MSE: linear regression - 2770663375546.279, catboost - 47105246474.34898
RMSE: linear regression - 1664530.9776469404, catboost - 217037.43104439147


## train on none repeating coords

In [10]:
print("number of columns:", len(manhatan_df.columns))
print("number of rows:", len(manhatan_df))
print("number of unique coord:", len(manhatan_df["coord"].unique()))

number of columns: 16
number of rows: 16007
number of unique coord: 5489


In [11]:
def statistics_of_repeating_coords(df, y_name):
    if not "coord_id" in df.columns:
        coord_2coordid = {coord:i for i,coord in enumerate(manhatan_df["coord"].unique())}
        manhatan_df["coord_id"] = manhatan_df.apply(lambda row: coord_2coordid[row["coord"]], axis=1)
    coord_y = df.loc[:, ["coord_id", y_name]]
    coord_statisics = coord_y.groupby(["coord_id"]).count()
    coord_statisics["std"] = coord_y.groupby(["coord_id"]).std()
    coord_statisics["mean"] = coord_y.groupby(["coord_id"]).mean()
    mean_std = coord_statisics.loc[coord_statisics[y_name]!=1, "std"].mean()
    df_mean = df[y_name].mean()
    print("mean std in price of appartment with the same coord:", mean_std)
    print("mean price in the dataset for comparison:", df_mean)
    coord_statisics = coord_statisics.rename(columns={y_name: "count"})
    return coord_statisics

In [12]:
statistics_of_repeating_coords(manhatan_df, "sold")

mean std in price of appartment with the same coord: 450228.5566634896
mean price in the dataset for comparison: 1600879.1584931593


,count,std,mean
coord_id,,,
0,11,189939.543611,8.558955e+05
1,1,NaN,8.250000e+05
2,19,190629.002690,5.775000e+05
3,17,671243.382704,1.273959e+06
4,1,NaN,8.250000e+05
...,...,...,...
5484,1,NaN,2.000000e+06
5485,1,NaN,1.090000e+06
5486,1,NaN,9.292880e+05


In [13]:
new_all_features = get_non_repeating_coords(all_features)
print("new number of rows", len(new_all_features))
training_cache = train_models(models=models, all_features=new_all_features, y_col=y_col)

new number of rows 5489
0:	learn: 1101839.1960303	total: 5.71ms	remaining: 851ms
1:	learn: 916844.3617910	total: 8.96ms	remaining: 663ms
2:	learn: 833759.9891231	total: 15.1ms	remaining: 740ms
3:	learn: 744108.0838727	total: 20.2ms	remaining: 737ms
4:	learn: 709334.4642934	total: 31.1ms	remaining: 903ms
5:	learn: 692833.7244635	total: 34.3ms	remaining: 822ms
6:	learn: 674554.9872386	total: 38.4ms	remaining: 784ms
7:	learn: 632421.7029766	total: 43.1ms	remaining: 765ms
8:	learn: 609348.1110509	total: 47.5ms	remaining: 745ms
9:	learn: 562085.4043964	total: 51.6ms	remaining: 723ms
10:	learn: 526161.8920559	total: 56.4ms	remaining: 713ms
11:	learn: 504610.2300983	total: 60.1ms	remaining: 691ms
12:	learn: 493897.7576074	total: 64.7ms	remaining: 681ms
13:	learn: 469728.3288113	total: 69.7ms	remaining: 677ms
14:	learn: 461917.7345289	total: 74ms	remaining: 666ms
15:	learn: 435613.8875320	total: 77.6ms	remaining: 650ms
16:	learn: 431382.1436564	total: 82ms	remaining: 641ms
17:	learn: 422865.54

In [14]:
plot_scores(training_cache)

mean price -  1848553.1427530954
MSE: linear regression - 4890466522380.548, catboost - 87441302278.8732
RMSE: linear regression - 2211439.920590326, catboost - 295704.75525238545
